<div class = "alert alert-block alert-success">

# <span style='color:Blue'> NOTEBOOK 2: Convert Training Data to a Tensorflow Record </span>

## Make sure you are running the *nightingale_env* kernel in this notebook

##  Using a Nightingale-formatted groundtruth CSV (created in Notebook-1), convert your training NITFs and annotations to a Tensorflow Record file. optionally including image chips that have no annotations (i.e., background only chips) for enhanced False Positive reduction. The steps in this notebook are: </span>
> ## 1. Specify the imagery folder (e.g., a folder of NITFs) and the groundtruth training CSV file
> ## 2. Make a list if NITFs/images that will be added to the Tensorflow Record for training the model 
> ## 3. Process the training data and add it to a Tensorflow Record

In [1]:
from data.io.tf_conversion_tools import *
import os
#os.environ['OPENCV_IO_MAX_IMAGE_PIXELS'] = pow(2,40).__str__() # if you are using PNGs instaed of NITFs and your png files are larger than cv2's default max size
import numpy as np
import cv2
import random
import pandas as pd
import tensorflow as tf
from osgeo.gdal import Open as gdalOpen
from libs.label_name_dict.label_dict import NAME_LABEL_MAP
from libs.configs.cfgs import DATASET_NAME, TFRECORD_PATH, TFRECORD_NAME, TRAIN_IMG_CHIP_SIZE, TRAIN_IMG_CHIP_OVRLP
print('Dataset name: ', DATASET_NAME)
print('Dataset categories including background: ', NAME_LABEL_MAP)
print('The Tensorflow Record will be saved as :', os.path.join(TFRECORD_PATH, TFRECORD_NAME))
print('The training image chip height & width will be', TRAIN_IMG_CHIP_SIZE, 'pixels')

<div class = "alert alert-block alert-warning">
    
## <span style='color:red'> ^ Check that your ROOT_PATH, DATASET_NAME,  label dictionary, & TFRECORD_PATH are set how you want them ^
    
## <span style='color:green'> Note for tutorial: We recommend that TRAIN_IMG_CHIP_SIZE (in you config file) be set just large enough for safely maximizing the GPU memory during network training (we found this value to be 2000 pixels for our Tesla V100 GPUs with 32478MiB of memory). But for the tutorial, you can reduce TRAIN_IMG_CHIP_SIZE to 1000 or smaller to speed up the training process. 

<div class = "alert alert-block alert-success">

## <span style='color:Blue'> 1) Specify the NITF directory and training-set groundtruth file and check the format of the groundtruth file

In [2]:
raw_images_dir = 'data/IMAGERY/'
gt_path = 'data/CSVs/OMITTED_TRAIN_Data_Nightingale_Format.csv'
gt_dataframe = pd.read_csv(gt_path)
print('groundtruth csv must be include the following columns:')
print("['IMID', 'xLF', 'yLF', 'xRF', 'yRF', 'xRB', 'yRB', 'xLB', 'yLB','class']")
print('Your groundtruth contains the following:')
print(gt_dataframe.columns)

groundtruth csv must be include the following columns:
['IMID', 'xLF', 'yLF', 'xRF', 'yRF', 'xRB', 'yRB', 'xLB', 'yLB','class']
Your groundtruth contains the following:
Index(['IMID', 'xLF', 'yLF', 'xRF', 'yRF', 'xRB', 'yRB', 'xLB', 'yLB',
       'class'],
      dtype='object')


<div class = "alert alert-block alert-success">

## <span style='color:Blue'> 2) Get the list of training images from the training-set groundtruth file, shuffle for training, and check that the number of training images and training image id's in groundtruth match. Note that this will only grab images that are included in the training-set csv file, so you don't need to manually separate your images into different folders. 

In [3]:
labels_id = list(np.unique(gt_dataframe['IMID'].to_numpy(dtype=np.str))) # unique list of image id's

images = []
labels = []
all_images = os.listdir(raw_images_dir)
random.shuffle(all_images)
for i in all_images:
    image_id = i.split('.')[0]
    if image_id in labels_id:
        images.append(i)
    

print('found training images:', len(images))
print('found training labels:', len(labels_id))

found training images: 2
found training labels: 2


<div class = "alert alert-block alert-success">

## <span style='color:Blue'> 3) Convert the training NITFs and training CSV annotations to a Tensorflow Record for training. Be sure to read the docstring for "format_image_label" and "data_2_tfrec" functions from data.io.tf_conversion_tools. Later, these two functions will allow you to retrain a model on imagery chips where False Positives were found in order to reduce FP occurences with the final model. 

In [2]:
#########################################
### Edit the following parameters #######
#########################################
image_format = 'nitf' # can be 'nitf' or 'png'
#########################################
####### Run the cell ####################
#########################################

img_h = TRAIN_IMG_CHIP_SIZE
img_w = TRAIN_IMG_CHIP_SIZE
stride_h = TRAIN_IMG_CHIP_SIZE-TRAIN_IMG_CHIP_OVRLP
stride_w = TRAIN_IMG_CHIP_SIZE-TRAIN_IMG_CHIP_OVRLP 

if not os.path.exists(TFRECORD_PATH):
    os.makedirs(TFRECORD_PATH)
save_path = os.path.join(TFRECORD_PATH, TFRECORD_NAME)
print('saving to ', save_path)
writer = tf.python_io.TFRecordWriter(path=save_path)

total_full_count = 0
total_empty_count = 0
PIXEL_MEAN = []
PIXEL_MEAN_ = []
PIXEL_STD = []
for idx, img in enumerate(images):
    
    img_id = img.split('.')[0]
    
    print(idx, 'reading image', img)
    
    if image_format == 'png':
        img_data = cv2.imread(os.path.join(raw_images_dir, img))
        bits = None
    elif image_format == 'nitf':
        my_nitf = gdalOpen(os.path.join(raw_images_dir, img))
        bits = int(my_nitf.GetMetadata()['NITF_ABPP'])
        #img_data = my_nitf.GetRasterBand(1).ReadAsArray()
        img_data = my_nitf.GetRasterBand(1).GetVirtualMemArray()
        
    
    #box = format_label(gt_dataframe,img_id,list(NAME_LABEL_MAP.keys()))
    box = format_image_label(gt_dataframe,img_id,NAME_LABEL_MAP)
    
    full_count, empty_count, pix_metrics = data_2_tfrec(file_idx = img.split('.')[0], image = img_data, 
                                         img_format = image_format, bits = bits,
                                         boxes_all = box, width = img_w, height = img_h, 
                                         stride_w = stride_w, stride_h = stride_h, 
                                         writer = writer, add_random_empties=False)
    PIXEL_MEAN.append(pix_metrics['pix_mean'])
    PIXEL_MEAN_.append(pix_metrics['pix_mean_normed'])
    PIXEL_STD.append(pix_metrics['pix_std_normed'])
    total_full_count += full_count
    total_empty_count += empty_count
    text = 'total chips w/ labels = '+str(total_full_count)+', total hard-negative (empty) chips = ', total_empty_count
    print(text)
PIXEL_MEAN = np.mean(np.array(PIXEL_MEAN),axis=0)
PIXEL_MEAN_= np.mean(np.array(PIXEL_MEAN_),axis=0)
PIXEL_STD = np.mean(np.array(PIXEL_STD),axis=0)
print('PIXEL_MEAN = ', list(PIXEL_MEAN))
print('PIXEL_MEAN_ = ', list(PIXEL_MEAN_))
print('PIXEL_STD = ', list(PIXEL_STD))
writer.close()

<div class = "alert alert-block alert-success">

## <span style='color:Blue'> You are done converting your data for training the model!
    
## <span style='color:Blue'> When the above cell finishes running, three variables are printed at the end (PIXEL_MEAN, PIXEL_MEAN_, and PIXEL_STD). These metircs were determiend while processing the imagery data. Update these variables in your cfgs.py file. 
    
## <span style='color:Blue'> If you set "add_random_empties=True" in the data_2_tfrec function, or if your groundtruth contained background category labels, then you will also see that the "total hard-negative (empty) chips" count is greater than zero. It's probably a bad idea to have more empty chips than chips with labels. Reconfigure some settings and redo the TF Record conversion if that's the case!

## <span style='color:Blue'> Doublecheck the settings in your cfgs.py file (in libs/configs/) and then run multi_train_gpu.py (in Training/tools/) from the terminal (see Notebook #3).